In [4]:
import numpy as np

In [210]:
def interval_count(curr, max_val, step, count):
    if curr >= max_val:
        return count
    else:
        if step(curr) <= 0:
            raise RuntimeError("Step function returns non-positive value in range")
        return interval_count(curr + step(curr), max_val, step, count + 1)

In [194]:
#This takes in the following data types:
# - x_min, x_max, y_min, y_max, z_min, z_max: Either a tuple (position, f(x_1, x_2)), where position is the x, y or z value of the side and the min value is less than the max val and 
#       f(x_1, x_2) is a function for the potential on that surface in terms of the other 2 variables, or a Nonetype value, representing a side going off to infinity
# - x_interval, y_interval, z_interval: These are functions that take in a real number and return a positive number in the range [v_min, v_max] for v=x,y,z. 
#       For a given value this function returns how far away in the correpsponding cartesian coordinate the next data point is, meaning if you want more points in one area, the interval
#       functions should return a smaller positive value in that area. Default values are functions that return 1, giving uniform scaling.
# - inf_val: this is a number of some kind, and represents the arbitrarily large number used to model an infinite side, default value is 2^20 ~ 10^6
def init_space(x_min, x_max, y_min, y_max, z_min, z_max, x_interval = lambda x: 1, y_interval = lambda y: 1, z_interval = lambda z: 1, inf_val = 2 ** 20):
    valnet = np.zeros((interval_count(x_min[0], x_max[0], x_interval, 1),
                      interval_count(y_min[0], y_max[0], y_interval, 1), 
                      interval_count(z_min[0], z_max[0], z_interval, 1)))
    
    posnet = np.zeros((interval_count(x_min[0], x_max[0], x_interval, 1),
                      interval_count(y_min[0], y_max[0], y_interval, 1), 
                      interval_count(z_min[0], z_max[0], z_interval, 1), 3))
    
    space = {"position": np.copy(posnet), "value": np.copy(valnet)}
    
    #Set the position vectors
    for x in range(len(space["position"])):
        for y in range(len(space["position"][0])):
            for z in range(len(space["position"][0, 0])):
                x_val = x_min[0] if x==0 else min(space["position"][x-1, y, z, 0] + x_interval(space["position"][x-1, y, z, 0]), x_max[0])
                y_val = y_min[0] if y==0 else min(space["position"][x, y-1, z, 1] + y_interval(space["position"][x, y-1, z, 1]), y_max[0])
                z_val = z_min[0] if z==0 else min(space["position"][x, y, z-1, 2] + z_interval(space["position"][x, y, z-1, 2]), z_max[0])
                space["position"][x, y, z] = [x_val, y_val, z_val]
                
    #Set x = x_min[0] and x=x_max[0] sides
    for y_i in range(len(space["position"][0])):
            for z_i in range(len(space["position"][0, 0])):
                space["value"][0, y_i, z_i] = x_min[1](space["position"][0, y_i, z_i, 1], space["position"][0, y_i, z_i, 2])
                space["value"][-1, y_i, z_i] = x_max[1](space["position"][-1, y_i, z_i, 1], space["position"][-1, y_i, z_i, 2])
                
    #Set y_min & y_max sides.
    #Note, this method of data storage means the edges and corners of the space get a bit funky because of overlap, but that's fine since they aren't used for future calculations 
    #and are known values, so you don't need them to be exact.
    for x_i in range(len(space["position"])):
        for z_i in range(len(space["position"][0, 0])):
            space["value"][x_i, 0, z_i] = y_min[1](space["position"][x_i, 0, z_i, 0], space["position"][x_i, 0, z_i, 2])
            space["value"][x_i, -1, z_i] = y_max[1](space["position"][x_i, -1, z_i, 0], space["position"][x_i, -1, z_i, 2])
            
    #Set z_min & z_max sides
    for x_i in range(len(space["position"])):
        for y_i in range(len(space["position"][0])):
            space["value"][x_i, y_i, 0] = z_min[1](space["position"][x_i, y_i, 0, 0], space["position"][x_i, y_i, 0, 1])
            space["value"][x_i, y_i, -1] = z_max[1](space["position"][x_i, y_i, -1, 0], space["position"][x_i, y_i, -1, 1])
            
    return space

In [221]:
test = init_space((-3, lambda y,z: y*z + 1), (1,lambda y,z: -3),(0, lambda x,z: x*z + 1), (5,lambda x,z: -2),(0, lambda x,y: x*y + 1), (4,lambda x,y: -1), x_interval = lambda x: 0.3)

In [222]:
test["position"]

array([[[[-3. ,  0. ,  0. ],
         [-3. ,  0. ,  1. ],
         [-3. ,  0. ,  2. ],
         [-3. ,  0. ,  3. ],
         [-3. ,  0. ,  4. ]],

        [[-3. ,  1. ,  0. ],
         [-3. ,  1. ,  1. ],
         [-3. ,  1. ,  2. ],
         [-3. ,  1. ,  3. ],
         [-3. ,  1. ,  4. ]],

        [[-3. ,  2. ,  0. ],
         [-3. ,  2. ,  1. ],
         [-3. ,  2. ,  2. ],
         [-3. ,  2. ,  3. ],
         [-3. ,  2. ,  4. ]],

        [[-3. ,  3. ,  0. ],
         [-3. ,  3. ,  1. ],
         [-3. ,  3. ,  2. ],
         [-3. ,  3. ,  3. ],
         [-3. ,  3. ,  4. ]],

        [[-3. ,  4. ,  0. ],
         [-3. ,  4. ,  1. ],
         [-3. ,  4. ,  2. ],
         [-3. ,  4. ,  3. ],
         [-3. ,  4. ,  4. ]],

        [[-3. ,  5. ,  0. ],
         [-3. ,  5. ,  1. ],
         [-3. ,  5. ,  2. ],
         [-3. ,  5. ,  3. ],
         [-3. ,  5. ,  4. ]]],


       [[[-2.7,  0. ,  0. ],
         [-2.7,  0. ,  1. ],
         [-2.7,  0. ,  2. ],
         [-2.7,  0. ,  3. ],
